In [1]:
import os
import glob

In [76]:
from labeller import Labeller
from product import Product
from ingredient import Ingredient
from prep import Prep
from item import Item

class Outlet:
    """
    Class representing all the data of an outlet

    Attributes:
        Name: String
        Owner: String
        Products: Product[]
    """

    def __init__(self, name, owner, products):
        self.name = name
        self.owner = owner
        self.products = products
    ##Getters
    def get_name(self):
        """
        Get the name of the outlet
        """
        return self.name
    def get_owner(self):
        return self.owner
    def get_products(self):
        return self.products
    def add_product(self, product):
        self.products.append(product)
    def add_products(self, filepath):
        labeller = Labeller(filepath, self.name)
        labeller.read_recipes()
        def create_item(item):
            idx = item["ItemId"].values[0]
            name = item["Description"].values[0]
            return Item(idx, name)
        def create_prep(prep):
            idx = prep["PrepId"].values[0]
            name = prep["Description"].values[0]
            prep_ingredients = labeller.ingredients.loc[labeller.ingredients['Recipe'] == idx]
            prep_ingredients = create_ingredients(prep_ingredients)
            return Prep(idx, name, prep_ingredients)
        def create_ingredients(ingredients):
            ingredient_list = []
            try:
                for _,row in ingredients.iterrows():
                    if row["IngredientId"].startswith("I"):
                        item_data = labeller.items.loc[labeller.items['ItemId'] == row["IngredientId"]]
                        item = create_item(item_data)
                        ingredient = Ingredient(item.name, row["Qty"], item)
                    elif row["IngredientId"].startswith("P"):
                        prep_data = labeller.preps.loc[labeller.preps['PrepId'] == row["IngredientId"]]
                        prep = create_prep(prep_data)
                        ingredient = Ingredient(prep.name, row["Qty"], prep)
                    elif row["IngredientId"].startswith("R"):
                        prod_data = labeller.products.loc[labeller.products['ProdId'] == row["IngredientId"]]
                        prod = create_product(prod_data['ProdId'].values[0])
                        ingredient = Ingredient(prod.name, row["Qty"], prod)
                    ingredient_list.append(ingredient)
            except Exception as e:
                print("Error in creating ingredients: ", e)
                print("Row: ", row)
                print("Ingredients: ", ingredients)
            return ingredient_list
        def create_product(prodID):
            #Data from Products Dataframe
            Prod_row = labeller.products.loc[labeller.products['ProdId'] == prodID]
            name = Prod_row['Description'].values[0]
            #Data from Ingredients Dataframe
            ingredients = labeller.ingredients.loc[labeller.ingredients['Recipe'] == prodID]
            ingredients = create_ingredients(ingredients)
            return Product(prodID, name, ingredients)
        for _,row in labeller.products.iterrows():
            self.products.append(create_product(row['ProdId']))
    def remove_product(self, product):
        self.products.remove(product)
    def __str__(self):
        return self.name + " (" + self.owner+ ")"
    

In [77]:
x = Outlet("Outlet1", "Owner1", [])

In [3]:
os.chdir("../../../")
filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "raw", "Gather", "*.oc"))
print(filepath_list)

['/Users/vivaanwadhwa/Documents/GitHub/CFFS/data/raw/Gather/ALL_gather_menu_july30.oc', '/Users/vivaanwadhwa/Documents/GitHub/CFFS/data/raw/Gather/missing_gather_items_15_july.oc', '/Users/vivaanwadhwa/Documents/GitHub/CFFS/data/raw/Gather/Gather_missing_items.oc.oc']
